**Imports**

In [3]:
# Imports
import pandas as pd
import random
import numpy as np

**Dataset imports**

In [4]:
train_df = pd.read_csv(r'training.csv')
test_df = pd.read_csv(r'testing.csv')
# Printing the column headings to better understand the features
print(train_df.columns.values)

['Duration' 'Distance' 'Pickup_longitude' 'Pickup_latitude'
 'Dropoff_latitude' 'Dropoff_latitude.1' 'Haversine' 'Pmonth' 'Pickup_day'
 'Pickup_hour' 'Pickup_minute' 'Pickup_weekday' 'Dropoff_month'
 'Dropoff_day' 'Dropoff_hour' 'Dropoff_minute' 'Dropoff_weekday' 'Temp'
 'Precip' 'Wind' 'Humid' 'Solar' 'Snow' 'GroundTemp' 'Dust']


In [5]:
len(train_df.columns.values)

25

In [6]:
from deap import gp, creator, base, tools, algorithms
import operator

In [63]:
def protected_div(x,y):
    if y == 0:
        return 1
    else:
        return x/y

In [64]:
pset = gp.PrimitiveSet("MAIN", 23)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protected_div,2)
# pset.addPrimitive(operator.pow, 2)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin, pset=pset)

c:\Users\david\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\david\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [65]:
# Toolbox setup
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [66]:
def eval_fitness(individual, dataset):
    func = toolbox.compile(expr=individual)
    #print("Function compiled for individual!")
    dataset.reset_index()

    raw_fitness = 0

    for index, row in dataset.iterrows():
        durations = row.get("Duration")
        distances = row.get("Distance")
        pickup_longs = row.get("Pickup_longitude")
        pickup_lats = row.get("Pickup_latitude")
        dropoff_longs = row.get("Dropoff_latitude.1")
        dropoff_lats = row.get("Dropoff_latitude")
        haversines = row.get("Haversine")
        pmonths = row.get("Pmonth")
        pdays = row.get("Pickup_day")
        phours = row.get("Pickup_hour")
        pmins = row.get("Pickup_minute")
        pweekdays = row.get("Pickup_weekday")
        ddays = row.get("Dropoff_day")
        dhours = row.get("Dropoff_hour")
        dmins = row.get("Dropoff_minute")
        dweekdays = row.get("Dropoff_weekday")
        temps = row.get("Temp")
        precips = row.get("Precip")
        winds = row.get("Wind")
        humids = row.get("Humid")
        solars = row.get("Solar")
        snows = row.get("Snow")
        groundtemps = row.get("GroundTemp")
        dusts = row.get("Dust")

        estimate = func(distances, pickup_longs, pickup_lats,
                        dropoff_longs, dropoff_lats, haversines,
                        pmonths, pdays, phours, pmins,
                        pweekdays, ddays, dhours, dmins, 
                        dweekdays, temps, precips, winds,
                        humids, solars, snows, groundtemps, dusts)
        
        actual = durations

        raw_fitness = raw_fitness + abs(actual - estimate)
    
    average_fitness = raw_fitness / len(dataset)
    #print("Inividual fitness: " + str(average_fitness))
    # using average fitness to reward overall good even if it has one or two weird cases
    return average_fitness,

In [67]:
def fitness_function(individual):
    # print("evaluating fitness...")
    # print("individual:")
    # print(type(individual))
    trip_sample = train_df.sample(100, ignore_index=True, random_state=10)

    # print(trip_sample.columns.values)

    # break into iteratable arrays
    durations = trip_sample.loc[:,"Duration"]
    distances = trip_sample.loc[:,"Distance"]
    pickup_longs = trip_sample.loc[:,"Pickup_longitude"]
    pickup_lats = trip_sample.loc[:,"Pickup_latitude"]
    dropoff_longs = trip_sample.loc[:,"Dropoff_latitude.1"]
    dropoff_lats = trip_sample.loc[:,"Dropoff_latitude"]
    haversines = trip_sample.loc[:,"Haversine"]
    pmonths = trip_sample.loc[:,"Pmonth"]
    pdays = trip_sample.loc[:,"Pickup_day"]
    phours = trip_sample.loc[:,"Pickup_hour"]
    pmins = trip_sample.loc[:,"Pickup_minute"]
    pweekdays = trip_sample.loc[:,"Pickup_weekday"]
    ddays = trip_sample.loc[:,"Dropoff_day"]
    dhours = trip_sample.loc[:,"Dropoff_hour"]
    dmins = trip_sample.loc[:,"Dropoff_minute"]
    dweekdays = trip_sample.loc[:,"Dropoff_weekday"]
    temps = trip_sample.loc[:,"Temp"]
    precips = trip_sample.loc[:,"Precip"]
    winds = trip_sample.loc[:,"Wind"]
    humids = trip_sample.loc[:,"Humid"]
    solars = trip_sample.loc[:,"Solar"]
    snows = trip_sample.loc[:,"Snow"]
    groundtemps = trip_sample.loc[:,"GroundTemp"]
    dusts = trip_sample.loc[:,"Dust"]
    print("Compiling function...")
    func = toolbox.compile(expr=individual)
    print("function compiled")
    fitness = 0
    for i in range(len(trip_sample)):
        # print("fitness eval loop run " + str(i))
        # PSUEDOCODE: individual_fitness = abs(correct_duration - calculated_duration)
        # PSEUDOCODE: fitness = fitness + individual_fitness
        sample_row = trip_sample.iloc[[i]]
        # print("sample row: " + str(type(sample_row)))
        sample_features = sample_row.drop("Duration",axis=1)
        # print("sample features: " + str(type(sample_features)))

        correct_duration = durations[i]
        # print("correct duration for item " + str(i) + " is " + str(correct_duration))
        calculated_duration = func(
            distances[i], pickup_longs[i], pickup_lats[i], dropoff_longs[i], dropoff_lats[i],
            haversines[i], pmonths[i], pdays[i], phours[i], pmins[i], pweekdays[i], ddays[i],
            dhours[i], dmins[i], dweekdays[i], temps[i], precips[i], winds[i], humids[i], solars[i],
            snows[i], groundtemps[i], dusts[i]
        )
        # print("calculated duration for item " + str(i) + " is " + str(calculated_duration))

        individual_fitness = abs(correct_duration - calculated_duration)
        fitness = fitness + individual_fitness
    # print("Fitness: " + str(fitness))
    return fitness,

In [68]:
train_sample = train_df.sample(10000, ignore_index=True, random_state=10)

toolbox.register("evaluate", eval_fitness, dataset=train_sample)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [61]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
# mstats.register("std", np.std)
mstats.register("min", np.min)
# mstats.register("max", np.max)

In [49]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, stats=stats_fit,
                                   halloffame=hof, verbose=True, ngen=20)

gen	nevals	avg    	min    
0  	100   	37784.9	16.2142
1  	77    	2678.41	17.8736
2  	87    	371.553	17.8736
3  	71    	1.1824e+06	17.2633
4  	73    	1025.49   	17.2443
5  	79    	140.511   	16.0434
6  	79    	93.123    	16.0434
7  	76    	29256.9   	15.8644
8  	83    	202.559   	15.8644
9  	76    	42.0846   	15.8644
10 	85    	729.267   	15.5   
11 	76    	2715.89   	15.6318
12 	63    	2648.58   	15.6318
13 	75    	378.49    	15.4647
14 	78    	112.352   	15.4647
15 	74    	5031.51   	15.4647
16 	77    	142.052   	15.4647
17 	68    	1553      	15.4647
18 	70    	80670.3   	15.4647
19 	81    	568.788   	15.4647
20 	85    	1557.08   	15.4647


In [50]:
print(hof)

[[<deap.gp.Primitive object at 0x0000022E049118B0>, <deap.gp.Primitive object at 0x0000022E049118B0>, <deap.gp.Terminal object at 0x0000022E0490FE40>, <deap.gp.Terminal object at 0x0000022E0490FFC0>, <deap.gp.Primitive object at 0x0000022E04911A40>, <deap.gp.Terminal object at 0x0000022E0490FDC0>, <deap.gp.Terminal object at 0x0000022E0490FDC0>]]


In [53]:
str(hof[0])

'add(add(ARG6, ARG10), mul(ARG5, ARG5))'

In [54]:
# clearly not complete, let's keep going

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, stats=stats_fit,
                                   halloffame=hof, verbose=True, ngen=20)

gen	nevals	avg    	min    
0  	0     	1557.08	15.4647
1  	83    	1492.42	15.4647
2  	86    	366.29 	15.4486
3  	78    	364.683	15.4486
4  	76    	123.605	15.4486
5  	84    	288.875	15.4486
6  	85    	3457.72	15.6318
7  	71    	163.342	15.4647
8  	78    	21350.1	15.3308
9  	78    	470.613	15.3308
10 	83    	185458 	15.3308
11 	83    	6710.17	14.8642
12 	89    	251.95 	14.8642
13 	77    	1373.78	15.3128
14 	83    	272.491	15.108 
15 	74    	142650 	14.866 
16 	79    	3573.86	14.8642
17 	73    	927.101	14.5814
18 	78    	1087.65	14.5814
19 	63    	1033.33	14.5814
20 	79    	107.026	14.1975


In [55]:
str(hof[0])

'add(add(ARG5, add(add(ARG10, ARG5), ARG5)), add(add(add(ARG5, ARG5), ARG5), ARG5))'

In [70]:
# clearly not complete, let's keep going -added protectedDiv

pop = toolbox.population(n=100)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, stats=stats_fit,
                                   halloffame=hof, verbose=True, ngen=20)

gen	nevals	avg    	min    
0  	100   	9505.74	18.1285
1  	80    	6479.81	18.0265
2  	82    	97.4854	18.0248
3  	75    	437.973	17.8764
4  	72    	135.336	17.3033
5  	78    	33.1965	17.7711
6  	73    	2300.4 	17.3033
7  	73    	5139.01	17.1685
8  	70    	306509 	17.173 
9  	85    	1030.4 	17.2633
10 	73    	38.3125	17.2633
11 	87    	164.529	17.2633
12 	83    	196.279	17.1564
13 	77    	3473.7 	16.2265
14 	85    	5677.78	16.5578
15 	72    	4941.48	17.0341
16 	80    	98.7644	16.0504
17 	77    	114598 	14.1107
18 	77    	208.773	14.1107
19 	76    	584.193	12.6898
20 	70    	592.367	12.1815


In [71]:
str(hof[0])

'add(mul(ARG20, protected_div(protected_div(ARG10, mul(mul(ARG8, protected_div(ARG1, ARG17)), ARG5)), ARG7)), protected_div(sub(ARG18, ARG0), sub(ARG5, ARG2)))'

In [72]:
# getting closer...?
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, stats=stats_fit,
                                   halloffame=hof, verbose=True, ngen=20)

gen	nevals	avg    	min    
0  	0     	592.367	12.1815
1  	83    	142.676	12.1815
2  	79    	806.678	12.1815
3  	84    	11049.1	12.1815
4  	76    	431.34 	12.1814
5  	86    	498.822	11.5923
6  	87    	156847 	10.4407
7  	81    	19298.4	10.433 
8  	73    	126.129	10.433 
9  	82    	86.3524	10.433 
10 	82    	197.87 	9.27182
11 	79    	72.8986	9.27182
12 	74    	196.705	9.27183
13 	79    	132057 	9.27183
14 	73    	217777 	9.14664
15 	77    	104.427	9.27184
16 	70    	34.894 	9.17636
17 	83    	1.33265e+13	9.17636
18 	71    	23196.6    	9.17636
19 	75    	1.33265e+13	9.17191
20 	78    	345.789    	9.17537


In [73]:
str(hof[0])

'add(mul(ARG20, protected_div(protected_div(ARG16, ARG7), add(protected_div(mul(ARG0, ARG22), protected_div(ARG5, ARG6)), mul(ARG14, add(ARG13, ARG2))))), protected_div(sub(mul(sub(ARG8, ARG6), sub(ARG6, ARG11)), ARG0), sub(sub(ARG10, ARG18), ARG2)))'

In [74]:
import pickle

In [75]:
with open("deap_with_div_pop.pkl", "wb") as deap_savefile:
    pickle.dump(pop, deap_savefile)

# saved with best of 9.147

In [76]:
# div seemed to help a lot...
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, stats=stats_fit,
                                   halloffame=hof, verbose=True, ngen=20)

gen	nevals	avg    	min    
0  	0     	345.789	9.17537
1  	87    	1.15319e+06	9.17474
2  	78    	232.265    	9.17338
3  	80    	31.8668    	9.17177
4  	80    	114.789    	9.16745
5  	82    	621.456    	9.16753
6  	80    	34338.9    	9.13052
7  	73    	17.5476    	9.13052
8  	79    	8577.4     	9.09184
9  	82    	1769.11    	9.13047
10 	72    	16.2175    	9.13047
11 	82    	3181.14    	9.12905
12 	78    	1743.75    	9.12612
13 	82    	36986.7    	9.12612
14 	81    	2148.56    	9.10348
15 	74    	85.4853    	9.08093
16 	86    	347.586    	9.08093
17 	80    	2.09266e+07	9.08019
18 	80    	833.977    	9.04315
19 	82    	269.43     	9.043  
20 	82    	2.31055e+06	9.04315


In [77]:
str(hof[0])

'add(mul(ARG20, protected_div(protected_div(protected_div(ARG6, mul(ARG5, protected_div(protected_div(sub(mul(sub(protected_div(add(ARG2, ARG9), mul(ARG7, mul(ARG20, add(ARG13, ARG2)))), ARG6), ARG6), ARG2), sub(sub(ARG10, ARG18), ARG2)), ARG7))), ARG7), ARG18)), protected_div(sub(sub(sub(sub(mul(ARG18, ARG5), add(ARG8, ARG3)), protected_div(ARG13, ARG14)), ARG18), ARG0), sub(sub(ARG10, ARG18), ARG2)))'

In [78]:
with open("deap_with_div_pop_2.pkl", "wb") as deap_savefile:
    pickle.dump(pop, deap_savefile)

# saved with best of 9.043